In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, , , Waiting, , Waiting)

In [2]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news1")
display(df)

StatementMeta(, , , Waiting, , Waiting)

SynapseWidget(Synapse.DataFrame, d91676f2-365a-484e-904c-2409ab11a5e9)

Import Synapse ML

In [3]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, , , Waiting, , Waiting)

In [4]:
# Import the model and configure the input and output columns
model =(AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, , , Waiting, , Waiting)

In [5]:
# Apply the model to our dataframe
result = model.transform(df)

StatementMeta(, , , Waiting, , Waiting)

In [6]:
display(result)

StatementMeta(, , , Waiting, , Waiting)

SynapseWidget(Synapse.DataFrame, 84f34b85-9782-44a0-9f85-1bc4392111e3)

In [7]:
# create Sentiment column 
from pyspark.sql.functions import col

sentiment_df =result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, , , Waiting, , Waiting)

In [8]:
display(sentiment_df)

StatementMeta(, , , Waiting, , Waiting)

SynapseWidget(Synapse.DataFrame, ed07783d-0588-44b9-962c-722de4527e79)

Drop Unwanted columns

In [9]:
sentiment_df_final = sentiment_df.drop("error","response")

StatementMeta(, , , Waiting, , Waiting)

In [10]:
display(sentiment_df_final)

StatementMeta(, , , Waiting, , Waiting)

SynapseWidget(Synapse.DataFrame, ddeb4249-d0e8-48bc-b5a6-0a7204a3ed23)

In [17]:
from pyspark.sql.functions import col, to_date
sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date(col("datePublished"), "dd-MMM-yyy"))

StatementMeta(, ee60961b-7f12-45b5-a859-847fe16c5284, 19, Finished, Available, Finished)

Type 1 Merge

In [12]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_sentiment_analysis'
    sentiment_df_final.write.format("delta").saveAsTable(table_name)
except AnalysisException:
    print("Table Already Exits")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f""" MERGE INTO {table_name} target_table
                   USING vw_sentiment_df_final source_view

                   ON source_view.url = target_table.url

                   WHEN MATCHED AND 
                   source_view.title <> target_table.title or
                   source_view.description <> target_table.description or
                   source_view.category <> target_table.category or
                   source_view.image <> target_table.image or
                   source_view.provider <> target_table.provider or
                   source_view.datePublished <> target_table.datePublished 

                   THEN UPDATE SET *

                   WHEN NOT MATCHED THEN INSERT *
                """)

StatementMeta(, , , Waiting, , Waiting)

Table Already Exits
